In [4]:
!pip install langchain>=0.1.17 openai>=1.13.3 langchain_openai>=0.1.6 transformers>=4.40.1 datasets>=2.18.0 accelerate>=0.27.2 sentence-transformers>=2.5.1 duckduckgo-search>=5.2.2 langchain_community
!CMAKE_ARGS="-DLLAMA_CUDA=on" pip install llama-cpp-python==0.2.69

  Using cached llama_cpp_python-0.2.69.tar.gz (42.5 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached diskcache-5.6.3-py3-none-any.whl.metadata (20 kB)
Using cached diskcache-5.6.3-py3-none-any.whl (45 kB)
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.69-cp311-cp311-linux_x86_64.whl size=55723657 sha256=b7fc81ec70735e2094c87ba588a9fe5811a21c28bf50da5ea1949cb5642fc0f5
  Stored in directory: /root/.cache/pip/wheels/e8/1b/ff/b4dba97fbd16e731705b262602ba8f3b672bf4bde54ea0c104
Successfully built llama-cpp-python


In [5]:
!wget https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-fp16.gguf

--2025-06-14 05:52:29--  https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-fp16.gguf
Resolving huggingface.co (huggingface.co)... 13.35.202.40, 13.35.202.34, 13.35.202.97, ...
Connecting to huggingface.co (huggingface.co)|13.35.202.40|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/41/c8/41c860f65b01de5dc4c68b00d84cead799d3e7c48e38ee749f4c6057776e2e9e/5d99003e395775659b0dde3f941d88ff378b2837a8dc3a2ea94222ab1420fad3?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27Phi-3-mini-4k-instruct-fp16.gguf%3B+filename%3D%22Phi-3-mini-4k-instruct-fp16.gguf%22%3B&Expires=1749883949&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0OTg4Mzk0OX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzQxL2M4LzQxYzg2MGY2NWIwMWRlNWRjNGM2OGIwMGQ4NGNlYWQ3OTlkM2U3YzQ4ZTM4ZWU3NDlmNGM2MDU3Nzc2ZTJlOWUvNWQ5OTAwM2UzOTU3NzU2NTliMGRkZTNmOTQxZDg4Z

## Basic Chain

In [8]:
from langchain import LlamaCpp

llm = LlamaCpp(
    model_path="/content/Phi-3-mini-4k-instruct-fp16.gguf.1",
    n_gpu_layers=-1,
    max_tokens=500,
    n_ctx=2048,
    seed=42,
    verbose=False
)

In [9]:
llm.invoke("What is 1 + 1 ?")

'\n\n<|assistant|> The answer to 1 + 1 is 2.\n\nThis question falls under basic arithmetic, a fundamental topic in mathematics that deals with numbers and the operations performed on them: addition being one of these operations where two or more quantities are combined into a larger amount.'

In [10]:
from langchain import PromptTemplate

template = """<s><|user|>
{input_prompt}<|end|>
<|assistant|>"""

prompt = PromptTemplate(
    template=template,
    input_variables=["input_prompt"]
)

In [11]:
basic_chain = prompt | llm

In [12]:
basic_chain.invoke(
    {
        "input_prompt":"Hi! My name is Falif. What is 1 + 1 ?"
    }
)

' Hello Falif! The answer to 1 + 1 is 2.'

## Multiple Chain

In [13]:
from langchain import LLMChain

template = """<s><|user|>
Create a title for a story about {summary}. Only return the title.<|end|>
<|assistant|>"""
title_prompt = PromptTemplate(template=template, input_variables=["summary"])
title = LLMChain(llm=llm, prompt=title_prompt, output_key="title")

<ipython-input-13-3958448897>:7: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  title = LLMChain(llm=llm, prompt=title_prompt, output_key="title")


In [14]:
title.invoke({"summary": "A girl that lost her mother."})

{'summary': 'A girl that lost her mother.',
 'title': ' "Lost in Grief: The Journey of a Girl Without Her Mother"'}

In [15]:
template = """<s><|user|>
Describe the main character of a story about {summary} with the title {title}.
Use only two sentences.<|end|>
<|assistant|>"""
character_prompt = PromptTemplate(template=template, input_variables=["summary","title"])
character = LLMChain(llm=llm, prompt=character_prompt, output_key="character")

In [16]:
template = """<s><|user|>
Create a story about {summary} with the title {title}. The main character is: {character}.
Only return the story and it cannot be longer than one paragraph.<|end|>
<|assistant|>"""
story_prompt = PromptTemplate(template=template, input_variables=["summary","title","character"])
story = LLMChain(llm=llm, prompt=story_prompt, output_key="story")

In [17]:
llm_chain = title | character | story

In [18]:
llm_chain.invoke("A girl that lost her mother")

{'summary': 'A girl that lost her mother',
 'title': ' "Finding Light Amidst Loss: The Journey of Emily After Her Mother\'s Departure"',
 'character': ' Emily is an introspective and resilient young woman, who struggles to cope with the profound loss of her mother while seeking solace in nature and creative outlets like painting and writing poetry. Throughout her journey, she learns to embrace self-love, healing, and finding strength within herself as a beacon of light amidst her grief.',
 'story': " Finding Light Amidst Loss: The Journey of Emily After Her Mother's Departure follows the introspective and resilient young woman, Emily, as she grapples with the profound loss of her mother. As a means to cope with her grief, Emily immerses herself in nature and creative endeavors such as painting landscapes and writing heartfelt poetry. Throughout this journey, she learns that self-love, healing, and inner strength are essential for finding light amidst the darkness of her loss. Along the

## Memory

In [19]:
basic_chain.invoke(
    {
        "input_prompt":"Hi! My name is Falif. What is 1 + 1 ?"
    }
)

' The answer to 1 + 1 is 2. This is a basic arithmetic operation, where you are adding one unit to another unit.'

In [20]:
basic_chain.invoke({"input_prompt": "What is my name >"})

" I'm unable to determine your name as I don't have the ability to access personal data or information. However, if you provide me with context or details about where this question may be coming from, I might be able to assist you better!"

In [24]:
template = """<s><|user|>Current conversation:{chat_history}

{input_prompt}<|end|>
<|assistant|>"""

prompt = PromptTemplate(
    template=template,
    input_variables=["input_prompt","chat_history"]
)

### ConversationBufferMemory

In [25]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history")

llm_chain = LLMChain(
    prompt=prompt,
    llm=llm,
    memory=memory
)

In [26]:
llm_chain.invoke({"input_prompt": "Hi, my name is Falif. What is 1 + 1 ?"})

{'input_prompt': 'Hi, my name is Falif. What is 1 + 1 ?',
 'chat_history': '',
 'text': " The sum of 1 + 1 is 2.\n\nHere's a brief explanation for clarity: When you add one unit to another unit, it results in two units altogether. This basic arithmetic operation is fundamental and widely applicable across various fields."}

In [27]:
llm_chain.invoke({"input_prompt": "What is my name ?"})

{'input_prompt': 'What is my name ?',
 'chat_history': "Human: Hi, my name is Falif. What is 1 + 1 ?\nAI:  The sum of 1 + 1 is 2.\n\nHere's a brief explanation for clarity: When you add one unit to another unit, it results in two units altogether. This basic arithmetic operation is fundamental and widely applicable across various fields.",
 'text': ' Your name is Falif.\n\nThis response directly answers the question about your name based on the previous part of the conversation where you introduced yourself as "Falif."\n\nNow, let\'s address any further inquiries or tasks you might have!'}

### ConversationBufferWindowMemory

In [28]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=2, memory_key="chat_history")

llm_chain = LLMChain(
    prompt=prompt,
    llm=llm,
    memory=memory
)

<ipython-input-28-1275254419>:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=2, memory_key="chat_history")


In [30]:
llm_chain.predict(input_prompt="Hi! My name is Falif and I am 23 years old. What is 1 + 1")

" Hello Falif, it's nice to meet you too! When we talk about simple arithmetic like addition, the answer to 1 + 1 is quite straightforward—it equals 2. Remember that these foundational skills are part of a broader range of knowledge and experiences that shape who you are beyond your age. How can I assist you further today?"

In [31]:
llm_chain.predict(input_prompt="What is 3 + 3 ?")

" Absolutely, Falif! When we add the numbers 3 and 3 together, the sum is 6. Just like with any basic math operation, it's always great to explore more complex concepts as you continue learning! How else may I assist you today?"

In [32]:
llm_chain.invoke({"input_prompt": "What is my name ?"})

{'input_prompt': 'What is my name ?',
 'chat_history': "Human: Hi! My name is Falif and I am 23 years old. What is 1 + 1\nAI:  Hello Falif, it's nice to meet you too! When we talk about simple arithmetic like addition, the answer to 1 + 1 is quite straightforward—it equals 2. Remember that these foundational skills are part of a broader range of knowledge and experiences that shape who you are beyond your age. How can I assist you further today?\nHuman: What is 3 + 3 ?\nAI:  Absolutely, Falif! When we add the numbers 3 and 3 together, the sum is 6. Just like with any basic math operation, it's always great to explore more complex concepts as you continue learning! How else may I assist you today?",
 'text': ' Your name is Falif, as mentioned in your previous statement: "Hello Falif, it\'s nice to meet you too!" Is there anything specific you would like to know or discuss further?'}

In [33]:
llm_chain.invoke({"input_prompt": "What is my age ?"})

{'input_prompt': 'What is my age ?',
 'chat_history': 'Human: What is 3 + 3 ?\nAI:  Absolutely, Falif! When we add the numbers 3 and 3 together, the sum is 6. Just like with any basic math operation, it\'s always great to explore more complex concepts as you continue learning! How else may I assist you today?\nHuman: What is my name ?\nAI:  Your name is Falif, as mentioned in your previous statement: "Hello Falif, it\'s nice to meet you too!" Is there anything specific you would like to know or discuss further?',
 'text': " I'm an AI and don't have access to personal data such as your age. However, I can help answer questions about general topics! If you need assistance with something else, feel free to ask."}

### Conversation Summary

In [41]:
summary_prompt_template = """<s><|user|>Summarize the conversations and update with the new lines.

Current summary:
{summary}

New lines of conversation:
{new_lines}

New summary:<|end|>
<|assistant|>"""
summary_prompt = PromptTemplate(
    template=summary_prompt_template,
    input_variables=["new_lines","summary"]
)

In [42]:
from langchain.memory import ConversationSummaryMemory

memory = ConversationSummaryMemory(llm=llm, memory_key="chat_history", prompt=summary_prompt)

llm_chain = LLMChain(
    prompt=prompt,
    llm=llm,
    memory=memory
)

In [43]:
llm_chain.invoke({"input_prompt": "Hi! My name is Muhammad Sya'bani Falif. What is 1 + 1 ?"})

{'input_prompt': "Hi! My name is Muhammad Sya'bani Falif. What is 1 + 1 ?",
 'chat_history': '',
 'text': " Hello Muhammad Sya'bani Falif, the sum of 1 + 1 is 2.\n\nIf you have any other questions or need further assistance, feel free to ask!"}

In [44]:
llm_chain.invoke({"input_prompt": "What is my name ?"})

{'input_prompt': 'What is my name ?',
 'chat_history': " New summary: Muhammad Sya'bani Falif inquired about the result of 1+1 and was informed that it equals 2; no additional queries were made.",
 'text': " As an AI, I don't have access to personal data such as names unless it has been shared with me during our conversation. Therefore, I cannot determine your name from the information provided in this interaction about a math question. If you'd like to share your name for further context or assistance, feel free to do so."}

In [45]:
llm_chain.invoke({"input_prompt": "What was the first question i asked ?"})

{'input_prompt': 'What was the first question i asked ?',
 'chat_history': " Muhammad Sya'bani Falif asked about the result of 1+1 and was told it equals 2; later, he inquired about his name but received information that as an AI, I don't have access to personal data unless shared during our conversation.",
 'text': ' The first question you asked was: "What is the result of 1+1?"'}

In [46]:
llm_chain.invoke({"input_prompt": "What was the result you given ?"})

{'input_prompt': 'What was the result you given ?',
 'chat_history': " Muhammad Sya'bani Falif initially inquired about the result of 1+1, which equals 2. He then asked for information regarding his name and received a response stating that as an AI, I don't have access to personal data unless shared during our conversation. Additionally, he later confirmed his initial question by asking what it was.",
 'text': " The result I provided initially when Muhammad Sya'bani Falif inquired about 1+1 was 2."}

In [48]:
memory.load_memory_variables({})

{'chat_history': " Muhammad Sya'bani Falif asked for the result of 1+1, which equals 2, and confirmed his initial question by inquiring again about the response. The AI reiterated that it doesn't have access to personal data unless shared during their conversation but provided the answer regarding the mathematical query."}

## Agent

In [ ]:
import os
from langchain_openai import ChatOpenAI

In [ ]:
os.environ["OPENAI_API_KEY"] = "MY_KEY"
openai_llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [ ]:
react_template = """Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Tought/Action/Action Input / Observation can repeat N times)
Thought: I now know the final answer
Final answer: the final answer the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}"""

prompt = PromptTemplate(
    template=react_template,
    input_variables=["tools","tool_names","input","agent_scratchpad"]
)

In [ ]:
from langchain.agents import load_tools, Tool
from langchain.tools import DuckDuckGoSearchResults

search = DuckDuckGoSearchResults()
search_tool = Tool(
    name="duckduck",
    description="A web search engine. Use this to as a search engine for general queries.",
    func=search.run
)

tools = load_tools(["llm-math"], llm=openai_llm)
tools.append(search_tool)

In [ ]:
from langchain.agents import AgentExecutor, create_react_agent

agent = create_react_agent(openai_llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

In [ ]:
agent_executor.invoke(
    {
        "input": "What is the current price of a MacBook Pro in USD? How much would it cost in EUR if the exchange rate is 0.85 EUR for 1 USD."
    }
)